In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### Import modules needed to analyze Deluxe dataset

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

## Loading and Understanding Deluxe dataset

In [ ]:
# Loading the dataset
df = pd.read_csv("/kaggle/input/online-store-sales-data/Sales-Export_2019-2020.csv")

# Dataset info
print(df.info())

# Dataset Shape
print(f"\nDeluxe dataset has {df.shape[0]} rows and {df.shape[1]} columns \n")

# Dataset Columns
print(f"The dataset has the following columns: \n{df.columns} \n" )

df.head()

## Data Cleaning

In [ ]:
# Removing spaces in column names
df.columns = df.columns.str.strip()

# Check duplicate rows
print(f"The sum of duplicted rows: {df.duplicated().sum()} \n")

# Count total nulls per column 
print(f"Total nulls: \n{df.isnull().sum()} \n")

# Rename 'order_value_EUR' to 'revenue.'
df.rename(columns = {"order_value_EUR":"revenue"}, inplace = True)

# Convert date from object type to datetime type
df["date"] = pd.to_datetime(df['date'])

# Remove ',' on revenue and convert the column from object to float type
df['revenue'] = df['revenue'].str.replace(",","", regex = True).astype('float')

df.info()

## Feature Engineering

In [ ]:
# Create new features, namely: profit, profit_margin, year, month, week_day
df['profit'] = round(df.revenue - df.cost,2)
df['profit_margin'] = round((df.profit / df.revenue)* 100, 2)
df['year'] = df['date'].dt.year
df['month_num'] = df['date'].dt.month
df['month'] = df['date'].dt.month_name()
df['week_day'] = df['date'].dt.day_name()

df.to_csv("Deluxe_Cleaned_data.csv", index = False)
df.head()

# 1. SALES ANALYSIS

Understand revenue drivers, seasonality, and growth

In [ ]:
# Aggregate revenue/order_value per year+month 
monthly_sales = df.groupby(['year','month_num','month'])['revenue'].sum().reset_index()

# Sort by year and month_num 
monthly_sales = monthly_sales.sort_values(by=['year','month_num']) 

plt.figure(figsize=(12,6)) 
sns.lineplot(data=monthly_sales, x='month', y='revenue', hue='year', marker='o', palette = 'coolwarm') 
plt.title("Monthly Sales Trend") 
plt.xlabel("Month") 
plt.ylabel("Revenue") 
plt.xticks(rotation=45) 
plt.show()

In [ ]:
# Revenue by category
category_revenue = df.groupby('category')['revenue'].sum().reset_index()
category_revenue = category_revenue.sort_values(by = 'revenue', ascending = False)
print(category_revenue.head())

plt.figure(figsize=(12,6))
sns.barplot(data = category_revenue, y = 'category', x = 'revenue', hue = 'category', palette = 'viridis')
plt.title('Revenue by category')
plt.xlabel("Revenue")
plt.ylabel('Category')
# plt.xticks(rotation = 45)
plt.show()

In [ ]:
#Revenue distribution per category

plt.figure(figsize = (12,6))
sns.boxplot(data = df, x="category", y="revenue")
plt.title("Revenue distribution per category")
plt.xlabel("Category")
plt.ylabel("Revenue")
plt.xticks(rotation = 45)
plt.show()

In [ ]:
# Revenue Distribution per Order
plt.figure(figsize = (12,6))
sns.histplot(df["revenue"], bins=30)
plt.title("Revenue Distribution per Order")
plt.xlabel("Revenue per Order")
plt.ylabel("Number of Orders")
plt.show()

# 2. Profitability and Cost Analysis

In [ ]:
# Profit Margin per Category

# Aggregate mean profit margin per category 
category_margin = df.groupby('category')['profit_margin'].mean().reset_index()
category_margin = category_margin.sort_values(by='profit_margin', ascending=False)

plt.figure(figsize = (12,6))
sns.barplot(data=category_margin, x="category", y="profit_margin",hue= 'category', palette = 'viridis')
plt.title("Profit Margin per Category")
plt.xlabel("Category")
plt.ylabel("Profit Margin (%)")
plt.show()

In [ ]:
# Cost vs Revenue

plt.figure(figsize = (12,6))
sns.scatterplot(data = df, x = 'cost', y = 'revenue', hue = 'category', palette = 'coolwarm')
plt.title('Cost vs Revenue')
plt.xlabel('Cost')
plt.ylabel('Revenue')
plt.show()

In [ ]:
# Profit Distribution

plt.figure(figsize = (12,6))
sns.histplot(df["profit"], bins=30)
plt.title('Profit Distribution')
plt.xlabel('Profit')
plt.ylabel('Frequency')
plt.show()


# 3. Product Category Analysis

In [ ]:
# Orders Count by Category

plt.figure(figsize=(12,8))
sns.countplot(data=df,x="category", hue = 'category', palette = 'viridis')
plt.title("Number of Orders per Product Category")
plt.xlabel("Product Category")
plt.ylabel("Number of Orders")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Category Seasonality

cat_monthly = df.groupby(['month_num','month', 'category'])['revenue'].sum().reset_index()
cat_monthly = cat_monthly.sort_values(by = 'month_num')

plt.figure(figsize=(12,8))
sns.lineplot(data=cat_monthly,x="month",y="revenue",hue="category")
plt.xlabel('Month')
plt.ylabel('Revenue')
plt.show()


# 4. Customer Analysis

In [ ]:
# Customer Lifetime Value (CLV) Distribution

customer_df = (df.groupby("customer_name")["revenue"].sum().reset_index())

plt.figure(figsize=(10,6))
sns.histplot(customer_df["revenue"], bins=50, kde=True)
plt.xlabel("Total Spend per Customer")
plt.ylabel("Number of Customers")
plt.title("Customer Lifetime Value Distribution")
plt.show()

In [ ]:
# Repeat vs One-Time Buyers

purchase_count = (df.groupby("customer_name")["order_id"].nunique().reset_index())

purchase_count["purchase_type"] = purchase_count["order_id"].apply(
    lambda x: "One-time Buyer" if x == 1 else "Repeat Buyer"
)

plt.figure(figsize=(12,6))
sns.countplot(data=purchase_count, x="purchase_type", hue = 'purchase_type', palette = 'viridis')
plt.title("Repeat vs One-time Buyers")
plt.xlabel("Customer Type")
plt.ylabel("Number of Customers")
plt.show()




# 5. Country / Market Analysis

In [ ]:
# Total Revenue by Country

# Group data by country and sum revenue
country_df = (df.groupby("country")["revenue"].sum().reset_index())
# Sort values for better visualization
country_df = country_df.sort_values(by="revenue",ascending=False)

plt.figure(figsize=(10, 5))
sns.barplot(data=country_df,x="country",y="revenue", hue = 'country', palette = 'magma')

plt.title("Total Revenue by Country")
plt.xlabel("Country")
plt.ylabel("Revenue")
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Profit Margin by Country

plt.figure(figsize=(12, 6))
sns.boxplot(data=df,x="country",y="profit_margin", hue = 'country', palette = 'viridis')
plt.title("Profit Margin Distribution by Country")
plt.xlabel("Country")
plt.ylabel("Profit Margin (%)")
plt.xticks(rotation=45)
plt.show()


# 6. Device Type Analysis

In [ ]:
# Device Revenue Distribution

plt.figure(figsize = (12,6))
sns.violinplot(data = df, x = 'device_type', y = 'revenue', hue = 'device_type', palette = 'viridis')
plt.title("Device Revenue Distribution")
plt.xlabel("Device Type")
plt.ylabel("Revenue")
plt.show()

In [ ]:
# Orders per Device

plt.figure(figsize = (12,6))
sns.countplot(data = df, x = 'device_type', hue = 'device_type', palette = 'viridis')
plt.title("Orders per Device")
plt.xlabel("Device Type")
plt.ylabel("Frequency")
plt.show()

# 7. Sales Team Performance

In [ ]:
# Revenue by Sales Representative

# Group data by sales representative and sum revenue

rep_total = df.groupby('sales_rep')['revenue'].sum().reset_index()
rep_total = rep_total.sort_values(by = 'revenue', ascending = False)

plt.figure(figsize= (12,15))
sns.barplot(data = rep_total, x = 'revenue', y = 'sales_rep', hue = 'sales_rep', palette = 'viridis')
plt.title('Revenue by Sales Representative')
plt.xlabel('Revenue')
plt.ylabel('Sales Representative')
plt.show()


In [ ]:
# Profit by Sales Manager

plt.figure(figsize=(10, 5))
sns.boxplot(data=df,x="sales_manager",y="profit", hue = 'sales_manager', palette = 'viridis')
plt.title("Profit Distribution by Sales Manager")
plt.ylabel("Profit per Order")
plt.xlabel("Sales Manager")
plt.xticks(rotation=75)
plt.show()


# 8. Time-Series and Trend Analysis

In [ ]:
# Moving Average Revenue

#daily total revenue
daily_sales = (df.groupby("date")["revenue"].sum().reset_index())

# Calculate Moving Average (7-day example)
daily_sales["moving_avg"] = (daily_sales["revenue"].rolling(window=7).mean())

plt.figure(figsize=(12, 6))
sns.lineplot(data=daily_sales,x="date",y="moving_avg")
plt.title("7-Day Moving Average of Revenue")
plt.xlabel("Date")
plt.ylabel("Revenue")
plt.show()



In [ ]:
# Sales Volatility

# Calculate Rolling Standard Deviation
daily_sales["std_dev"] = (daily_sales["revenue"].rolling(window=7).std())

plt.figure(figsize=(12, 6))

sns.lineplot(data=daily_sales,x="date",y="std_dev")

plt.title("7-Day Sales Volatility")
plt.xlabel("Date")
plt.ylabel("Standard Deviation of Revenue")
plt.show()

# 9. Basket and Order Behavior

Basket & Order Behavior analysis focuses on how much customers spend per order, rather than who or what they buy. 
This helps identify:
* High-value orders

* Upsell opportunities

* Pricing strategy improvements

In [ ]:
# Order Value Segmentation by Category

plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x="category", y="revenue", hue = 'category', palette = 'viridis')
plt.title("Order Value Distribution by Product Category")
plt.xlabel("Categories")
plt.ylabel("Revenue")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# High-Value Order Identification
plt.figure(figsize=(12, 5))
sns.scatterplot(data=df, x="order_id", y="revenue")
plt.title("Order Value per Order")
plt.xlabel("Order ID")
plt.ylabel("Revenue")
plt.show()

# 10. Executive KPI Dashboard

In [ ]:
# Revenue vs Profit Analysis

kpi_df = (df.groupby("category")[["revenue", "profit"]].sum().reset_index())


plt.figure(figsize = (12,6))
sns.scatterplot(data = kpi_df, x = 'revenue', y = 'profit', s = 120)
plt.title('Revenue vs Profit Analysis')
plt.xlabel('Revenue')
plt.ylabel('Profit')
plt.show()